# Update Summary Report HTML version

## Prerequsite 

In [ ]:

# SET log level
# logging
import logging
logger = logging.getLogger()
logger.setLevel(logging.WARN)
logging.info("Prerequisite")

import ipywidgets as widgets
from IPython.display import display, HTML
import os
import configparser
config = configparser.ConfigParser()
config.read('data.conf')

# ElasticSearch
es_host = config.get("data", 'es_host') 
es_port = config.get("data", 'es_port')
es_user = config.get("data", 'es_user') 
es_password = config.get("data", 'es_password')

# Grafana
os.environ['grafana_url'] = config.get("data", 'grafana_url')
os.environ['network_speed'] = config.get("data", 'network_speed')
os.environ['fetch_ocp_versions_days'] = config.get("data", 'fetch_ocp_versions_days')
os.environ['filter_kind'] = config.get("data", 'filter_kind')

# AWS
endpoint_url = config.get("data", 'service_name') 
region_name = config.get("data", 'region_name') 
endpoint_url= config.get("data", 'endpoint_url') 
aws_access_key_id = config.get("data", 'aws_access_key_id') 
aws_secret_access_key = config.get("data", 'aws_secret_access_key') 
bucket = config.get("data", 'bucket') 
key = config.get("data", 'key')

# Google drive
google_folder_id = config.get("data", 'google_folder_id')

from benchmark_runner.common.google_drive.google_drive_operations import GoogleDriveOperations
from benchmark_runner.common.elasticsearch.elasticsearch_operations import ElasticSearchOperations
from benchmark_runner.jupyterlab.templates.summary_report.summary_report_widgets import SummaryReportWidgets

elasticsearch = ElasticSearchOperations(es_host=es_host, es_port=es_port, es_user=es_user, es_password=es_password)
summary_report_widget = SummaryReportWidgets(elasticsearch=elasticsearch)

# Default 365 days for ocp version fetch days 
summary_report_widget.set_fetch_ocp_version_days(num_fetch_days=int(os.environ['fetch_ocp_versions_days']))


## Update file name with 2 last versions comparison

In [ ]:
import shutil

# Get the two last major versions
versions = summary_report_widget.get_two_last_major_versions()

# Define the source and destination file paths
source = 'summary_report.html'
destination = f'summary_report_{versions[0]}_{versions[1]}.html'

# Copy the file
shutil.copyfile(source, destination)


## Upload summary report to Google drive (override)

In [ ]:
# Performance & Scale -> CNV -> Summary Report
shared_drive_id = google_folder_id
file_path = destination
gdrive_operations = GoogleDriveOperations()
gdrive_operations.upload_file(file_path, shared_drive_id)

## Generate html report manually - done by ci_pod

!pip install jupyter nbconvert

!/usr/local/bin/jupyter nbconvert --to html --execute /notebooks/templates/summary_report/summary_report.ipynb